In [95]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from scipy.stats import boxcox

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'train.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the training data file
train = pd.read_csv(file_content_stream, sep = '|')
train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [96]:
## reading the test file
file_key= 'test.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the training data file
test = pd.read_csv(file_content_stream, sep = '|')
test.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,4,467,88.48,4,8,4,0.014989,0.189465,0.571429
1,3,1004,58.99,7,6,1,0.026892,0.058755,0.259259
2,1,162,14.00,4,5,4,0.006173,0.086420,4.000000
3,5,532,84.79,9,3,4,0.026316,0.159380,0.642857
4,5,890,42.16,4,0,0,0.021348,0.047371,0.210526


## feature engineering using boxcox transformation
***

In [97]:
## feature engineering using boxcox transformation
transformed_trust_level = boxcox(train['trustLevel'])

train['trustLevel'] = transformed_trust_level[0]

transformed_trust_level = boxcox(test['trustLevel'])

test['trust_level'] = transformed_trust_level[0]

In [98]:
## grand total / totalScanTimeInSeconds
train['grandtotal_totalscantime'] = train['grandTotal'] / train['totalScanTimeInSeconds']

test['grandtotal_totalscantime'] = test['grandTotal'] / test['totalScanTimeInSeconds']

In [99]:
## scansWithoutRegistration * quantityModifications
train['scans_quantity'] = train['scansWithoutRegistration'] * train['quantityModifications']

test['scans_quantity'] = test['scansWithoutRegistration'] * test['quantityModifications']

In [100]:
## log base 10 of grandTotal
train['grandTotal_log10'] = np.log10(train['grandTotal'])

test['grandTotal_log10'] = np.log10(test['grandTotal'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [101]:
## 1 / valuePerSecond
train['1_valuePerSecond'] = 1 / train['valuePerSecond']

test['1_valuePerSecond'] = 1 / test['valuePerSecond']

In [102]:
## scannedLineItemsPerSecond ^ 2
train['scannedLineItemsPerSecond_2'] = train['scannedLineItemsPerSecond']**2

test['scannedLineItemsPerSecond_2'] = test['scannedLineItemsPerSecond']**2

In [103]:
## scannedLineItemsPerSecond * valuePerSecond
train['scannedLineItemsPerSecond_valuePerSecond'] = train['scannedLineItemsPerSecond'] * train['valuePerSecond']

test['scannedLineItemsPerSecond_valuePerSecond'] = test['scannedLineItemsPerSecond'] * test['valuePerSecond']

In [104]:
## trustLevel * scannedLineItemsPerSecond
train['interaction_1'] = train['trustLevel'] * train['scannedLineItemsPerSecond']

test['interaction_1'] = test['trustLevel'] * test['scannedLineItemsPerSecond']

## scannedLineItemsPerSecond_2 * trustLevel
train['interaction_2'] = train['scannedLineItemsPerSecond_2'] * train['trustLevel']

test['interaction_2'] = test['scannedLineItemsPerSecond_2'] * test['trustLevel']

## scannedLineItemsPerSecond_2 * scannedLineItemsPerSecond
train['interaction_3'] = train['scannedLineItemsPerSecond_2'] * train['scannedLineItemsPerSecond']

test['interaction_3'] = test['scannedLineItemsPerSecond_2'] * test['scannedLineItemsPerSecond']


In [105]:
## engineering feature based on the decision tree results

train['interaction_4'] = np.where(((train['trustLevel'] <= 0.431) &
                                      (train['scannedLineItemsPerSecond'] <= 0.012) &
                                      (train['totalScanTimeInSeconds'] <= 895)), 1, 0)

test['interaction_4'] = np.where(((test['trustLevel'] <= 0.431) &
                                      (test['scannedLineItemsPerSecond'] <= 0.012) &
                                      (test['totalScanTimeInSeconds'] <= 895)), 1, 0)


train['interaction_5'] = np.where(((train['trustLevel'] <= 0.431) &
                                      (train['trustLevel'] <= 1.212) &
                                      (train['totalScanTimeInSeconds'] <= 1304)), 1, 0)
test['interaction_5'] = np.where(((test['trustLevel'] <= 0.431) &
                                      (test['trustLevel'] <= 1.212) &
                                      (test['totalScanTimeInSeconds'] <= 1304)), 1, 0)

In [106]:
## defining input and target variables
x = train.drop(columns = ['fraud'])
y = train['fraud']

In [107]:
## scaling the data
scaler = MinMaxScaler()

x = pd.DataFrame(scaler.fit_transform(x))

In [108]:
train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,grandtotal_totalscantime,scans_quantity,grandTotal_log10,1_valuePerSecond,scannedLineItemsPerSecond_2,scannedLineItemsPerSecond_valuePerSecond,interaction_1,interaction_2,interaction_3,interaction_4,interaction_5
0,2.728286,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0,0.051898,0,1.737987,19.268739,0.000757,0.001428,0.075067,0.002065,2.082918e-05,0,0
1,1.561821,108,27.36,5,2,4,0.129630,0.253333,0.357143,0,0.253333,8,1.437116,3.947368,0.016804,0.032840,0.202458,0.026245,2.178276e-03,0,0
2,1.561821,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0,0.041003,50,1.793511,24.388674,0.000074,0.000352,0.013393,0.000115,6.305688e-07,0,0
3,3.240277,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0,0.051541,16,1.965249,19.402015,0.000262,0.000835,0.052467,0.000850,4.245289e-06,0,0
4,2.728286,430,81.53,3,7,2,0.062791,0.189605,0.111111,0,0.189605,14,1.911317,5.274132,0.003943,0.011905,0.171311,0.010757,2.475631e-04,0,0


In [109]:
x.columns = [['trustLevel', 'totalScanTimeInSeconds', 'grandTotal', 'lineItemVoids',
              'scansWithoutRegistration', 'quantityModifications', 'scannedLineItemsPerSecond',
              'valuePerSecond', 'lineItemVoidsPerPosition', 'grandtotal_totalscantime', 'scans_quantity', 
              'grandTotal_log10', '1_valuePerSecond', 'scannedLineItemsPerSecond_2', 'scannedLineItemsPerSecond_valuePerSecond', 
              'interaction_4', 'interaction_5', 'interaction_1', 'interaction_2', 'interaction_3']]
              

### Recursive Feature Elimination (RFE)
***

In [ ]:
## creating lists to store the results
logit_results = list()
rf_results = list()
ada_results = list()

## creating loop to run RFE 100 times
for i in range(0,100):
    
    print(i)
    
    ## splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)
    
    
    ## creating logitstic regression model for RFE
    logit_rfe = RFE(estimator = LogisticRegression(), n_features_to_select = 5).fit(x_train, y_train)
    rf_rfe = RFE(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3), 
                                                n_features_to_select = 5).fit(x_train, y_train)
    ada_rfe = RFE(estimator = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 3), 
                                                n_estimators = 500, learning_rate = 0.01)).fit(x_train, y_train)
    
    ## extracting rankings
    logit_results.append(logit_rfe.support_)
    rf_results.append(rf_rfe.support_)
    ada_results.append(ada_rfe.support_)

    
## turning lists into dataframes
rf_results = pd.DataFrame(rf_results)
rf_results.columns = x_train.columns

logit_results = pd.DataFrame(logit_results)
logit_results.columns = x_train.columns

ada_results = pd.DataFrame(ada_results)
ada_results.columns = x_train.columns

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
##
rf = (100*rf_results.apply(np.sum, axis = 0)) / (rf_results.shape[0])
logit = (100*logit_results.apply(np.sum, axis = 0)) / (logit_results.shape[0])
ada = (100*ada_results.apply(np.sum, axis = 0)) / (ada_results.shape[0])

In [ ]:
rf

In [ ]:
logit

In [ ]:
ada

In [ ]:
## combing all three dataframes
result = pd.concat([rf, logit, ada], axis=1, join="inner")

In [ ]:
## averaging across rows
result['average'] = result.apply(np.mean, axis = 1)
result

In [ ]:
## ranking features based on the average
result = result.sort_values(by = 'average', ascending = False)
result

In [132]:
## visualizing results
'''fig, axes = plt.subplots(3, 2, figsize = (16, 22))

axes[0,0].barh(logit.index, logit.values)
axes[0,0].set_xlabel('Feature Selection Percentage')
axes[0,0].set_title('Linear Regression')

axes[1,0].barh(rf.index, rf.values)
axes[1,0].set_xlabel('Feature Selection Percentage')
axes[1,0].set_title('Random Forest')

axes[2,0].barh(ada.index, ada.values)
axes[2,0].set_xlabel('Feature Selection Percentage')
axes[2,0].set_title('AdaBoost')'''

"fig, axes = plt.subplots(3, 2, figsize = (16, 22))\n\naxes[0,0].barh(logit.index, logit.values)\naxes[0,0].set_xlabel('Feature Selection Percentage')\naxes[0,0].set_title('Linear Regression')\n\naxes[1,0].barh(rf.index, rf.values)\naxes[1,0].set_xlabel('Feature Selection Percentage')\naxes[1,0].set_title('Random Forest')\n\naxes[2,0].barh(ada.index, ada.values)\naxes[2,0].set_xlabel('Feature Selection Percentage')\naxes[2,0].set_title('AdaBoost')"